In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2
# from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
mannequin_path = os.path.join('../','eventsData/training/mannequin')
# not_mannequin_path = os.path.join('../','not_mannequin_filter')

mannequin_img_path = [os.path.join(mannequin_path,x) for x in os.listdir(mannequin_path)]
# not_mannequin_img_path = [os.path.join(not_mannequin_path,x) for x in os.listdir(not_mannequin_path)]

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

random.shuffle(mannequin_img_path)
random.shuffle(mannequin_img_path)
# random.shuffle(not_mannequin_img_path)
# random.shuffle(not_mannequin_img_path)

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

In [ ]:
train_mannequin_img, test_mannequin_img = train_test_split(mannequin_img_path,test_size = 0.20,random_state=42)

In [ ]:
train_datagen = IDG(preprocessing_function=preprocess_input)
val_datagen = IDG(preprocessing_function=preprocess_input)
test_datagen = IDG(preprocessing_function=preprocess_input)

train_path = '../eventsData_train'
val_path = '../eventsData_val'
test_path = '../eventsData_test'
shape = (224,224,3)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,seed=33)
val_generator = val_datagen.flow_from_directory(val_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical',shuffle=True,seed=33)
test_generator = test_datagen.flow_from_directory(test_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='categorical')


In [ ]:
efficientB2_model = MobileNetV2(include_top=False,weights='imagenet',input_shape=shape)
efficientB2_model.trainable=False
x = GlobalAveragePooling2D()(efficientB2_model.output)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2,activation='softmax')(x)

model = Model(efficientB2_model.input,x)
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,epochs=20,validation_data=val_generator)

In [ ]:
model.save('mobilenetV2_softmax.h5')

In [ ]:
# from tensorflow.keras.models import load_model
# model_import = load_model('mobilenetV2.h5')